# Latest dataset exploration - n=131
This notebook focuses on the implementation of the Cob class, which encompasses the analysis developed as part of the initial exploration of 15min sampled data as the output of Isabella Dagen's insulin-need project preprocessing. It uses the Cob class from cob_analysis.py to analyse the data and create peak features to investigate the existence of diurnal patterns in the data.

In [15]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from src.cob_analysis import Cob
from src.config import INTERIM_DATA_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Previous preprocessing - 15 min sampled data based on earlier limited dataset
cob = Cob()
cob.read_interim_data(file_name='15min_iob_cob_bg',
                      file_type='csv',
                      sampling_rate=15)

Number of records: 599360
Number of people: 116
Systems used: ['OpenAPS']
Categories (1, object): ['OpenAPS']


In [16]:
# Produce a similar 15min resampled dataset from the new interim consolidated data

df = pd.read_parquet(INTERIM_DATA_DIR / 'device_status_tz_naive_df.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14845706 entries, 0 to 14845705
Data columns (total 33 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   system                            object        
 1   id                                string        
 2   created_at                        datetime64[ns]
 3   openaps/iob/iob                   Float32       
 4   openaps/iob/activity              object        
 5   openaps/iob/basaliob              object        
 6   pump/clock                        datetime64[ns]
 7   pump/status/timestamp             datetime64[ns]
 8   pump/status/status                object        
 9   device                            object        
 10  bg                                Float32       
 11  openaps/enacted/reason            object        
 12  iob                               Float32       
 13  openaps/enacted/rate              Float32       
 14  cob             